In [1]:
banana <- function(x){
   100*(x[2]-x[1]^2)^2+(1-x[1])^2
 }

In [2]:
dx1= function(x){(-400*x[1]*(x[2]-x[1]^2))-2*(1-x[1])}
dx2 = function(x){200*(x[2]-x[1]^2)}
grad<-function(x) return(c(dx1(x),dx2(x)))

In [43]:
fixed_step <- function (x0, f, g = NULL, info = FALSE,
                           maxiter = 100000000, tol = 1e-7) {
    eps <- 1e-7
    if (! is.numeric(x0))
        stop("Argument 'x0' must be a numeric vector.")
    n <- length(x0)

    # User provided or numerical gradient
    f <- match.fun(f)
    g<- match.fun(grad)


    x <- x0
    k <- 1
    fixstep <- 0.001
    while (k <= maxiter) {
        f1 <- f(x) #banana function
        g1 <- g(x) #gradient of the banana
        z1 <- sqrt(sum(g1^2)) #norm of the gradient of the banana
        if (z1 == 0) {
            warning(
                paste("Zero gradient at:", x, f1, "-- not applicable.\n"))
            return(list(xmin = x, fmin = f(x), niter = k))
        }
        # else use gradient vector
        g2 <- g1 / z1

        fixstep=0.001; f3 <- f(x - fixstep*g2) #one step ahead to make sure f3 is the lowest before going to the next x value

        # Find a minimum and know when to stop
        if (f3>=f1 && ((abs(f3-f1)<tol)  || (sqrt(sum(x-(x-fixstep*g2))^2))<tol)) {
            f3 <- f1
            return(list(xmin = x, fmin = f(x), niter = k))
        }
        x <- x - fixstep*g2
        #therefore, x starts over again, and the new x will be put into the function to ensure banana stays minimum
        k <- k + 1
    }
    if(k > maxiter)
        warning("Maximum number of iterations reached.\n")
    return(list(xmin = x, fmin = f(x), niter = k))
}

In [44]:
fixed_step(c(4,4),banana)

$xmin
[1] 1.011228 1.021497

$fmin
[1] 0.0002436161

$niter
[1] 16628

In [31]:
steep_descent <- function (x0, f, g = NULL, info = FALSE,
                           maxiter = 100000000, tol = .Machine$double.eps^(1/2)) {
    eps <- .Machine$double.eps^(1/2)
    if (! is.numeric(x0))
        stop("Argument 'x0' must be a numeric vector.")
    n <- length(x0)

    # User provided or numerical gradient
    f <- match.fun(f)
    g <- match.fun(grad)


    if (info) cat(0, "\t", x0, "\n")

    x <- x0
    k <- 1
    while (k <= maxiter) {
        f1 <- f(x)
        g1 <- g(x)
        z1 <- sqrt(sum(g1^2))
        if (z1 == 0) {
            warning(
                paste("Zero gradient at:", x, f1, "-- not applicable.\n"))
            return(list(xmin = NA, fmin = NA, niter = k))
        }
        # else use gradient as unit vector
        g1 <- g1 / z1

        a1 <- 0
        a3 <- 0.01; f3 <- f(x - a3*g1)

        # Find a minimum on the gradient line (or line search)
        while (f3 >= f1) {
            a3 <- a3/2; f3 <- f(x - a3*g1)
            if (a3 < tol/2 && ((abs(f3-f1)<tol)  || (sqrt(sum(x-(x-fixstep*g2))^2))<tol)) {
                x[x < eps] <- 0
                return(list(xmin = x, fmin = f(x), niter = k))
            }
        }

        x <- x - a3*g1
        if (info) cat(k, "\t", x, "\n")
        k <- k + 1
    }
    if(k > maxiter)
        warning("Maximum number of iterations reached -- not converged.\n")
    return(list(xmin = NA, fmin = NA, niter = k))
}

In [32]:
steep_descent(c(4,4),banana)

$xmin
[1] 1 1

$fmin
[1] 1.047892e-19

$niter
[1] 31455

In [36]:
oring=read.table("o_ring_data.txt",header=T)
loglik = function(x) {sum((1-oring$Failure)*(-x[1]-x[2]*oring$Temp)-log(1+exp(-x[1]-x[2]*oring$Temp)))}
loglikdx1<-function(x) {sum((exp(-x[1]-x[2]*oring$Temp)/(1+exp(-x[1]-x[2]*oring$Temp)))+oring$Failure-1)}
loglikdx2<-function(x) {sum(((oring$Temp*(exp(-x[1]-x[2]*oring$Temp)))/(1+exp(-x[1]-x[2]*oring$Temp)))
                            -((1-oring$Failure)*oring$Temp))}
dxloglik<-function(x) return(c(loglikdx1(x),loglikdx2(x)))

In [39]:
steep_ascent <- function (x0, f, g = NULL, info = FALSE,
                           maxiter = 100000000, tol = 1e-7) {
    eps <- 1e-7
    if (! is.numeric(x0))
        stop("Argument 'x0' must be a numeric vector.")
    n <- length(x0)

    # User provided or numerical gradient
    f <- match.fun(f)
    g <- match.fun(dxloglik)

    if (info) cat(0, "\t", x0, "\n")

    x <- x0
    k <- 1
    while (k <= maxiter) {
        f1 <- f(x)
        g1 <- g(x)
        z1 <- sqrt(sum(g1^2))
        if (z1 == 0) {
            warning(
                paste("Zero gradient at:", x, f1, "-- not applicable.\n"))
            return(list(xmin = NA, fmin = NA, niter = k))
        }
        # else use gradient as unit vector
        g1 <- g1 / z1

        a1 <- 0
        a3 <- 0.01; f3 <- f(x + a3*g1)

        # Find a minimum on the gradient line (or line search)
        while (f3 <= f1) {
            a3 <- a3/2; f3 <- f(x + a3*g1)
            if (a3 < tol/2) {
                if (info)
                    cat("Method of steepest descent converged to:", x, "\n")
                x[x < eps] <- 0
                return(list(xmin = x, fmin = f(x), niter = k))
            }
        }

        x <- x + a3*g1
        if (info) cat(k, "\t", x, "\n")
        k <- k + 1
    }
    if(k > maxiter)
        warning("Maximum number of iterations reached -- not converged.\n")
    return(list(xmin = NA, fmin = NA, niter = k))
}

In [40]:
steep_ascent(c(4,4),loglik)

$xmin
[1] 15.03984  0.00000

$fmin
[1] -240.6375

$niter
[1] 3245964